In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import nltk
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import data

In [ ]:
url_data = os.path.join('/kaggle/input', 'netflix-shows/netflix_titles.csv')
df = pd.read_csv(url_data)
df.head()

In [ ]:
df = df[['type', 'title', 'director', 'cast', 'country', 'rating', 'listed_in', 'description']]
df = df.fillna('other')
df = df.set_index('title')
data = df.copy()

In [ ]:
data.country.unique()

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
remove_word = set(stopwords.words('english') + list(punctuation))

df["type"] = df.type.str.lower()
df["director"] = df.director.str.lower()
df["cast"] = df.cast.str.lower()
df["country"] = df.country.str.lower()
df["rating"] = df.rating.str.lower()
df["listed_in"] = df.listed_in.str.lower()
df["description"] = df.description.str.lower()

df["type"] = df.type.apply(lambda x : ''.join(x.split()))
df["director"] = df.director.apply(lambda x : ''.join(x.split()))
df["cast"] = df.cast.apply(lambda x : ' '.join([''.join(i.split(' ')) for i in x.split(', ')]))
df["country"] = df.country.apply(lambda x : ''.join(x.split(', ')))
df["rating"] = df.rating.apply(lambda x : ''.join(x.split('-')))
df["listed_in"] = df.listed_in.apply(lambda x : [i for i in word_tokenize(x) if i not in remove_word])
df["description"] = df.description.apply(lambda x : [i for i in word_tokenize(x) if i not in remove_word])

In [ ]:
df

In [ ]:
df["listed_in"] = df.listed_in.apply(lambda x : ' '.join(x))
df["description"] = df.description.apply(lambda x : ' '.join(x))

In [ ]:
# df["bag_of_words"] = df.apply(lambda x : x.type + " " + x.director + " " + x.cast + " " + x.country + " " + x.rating + " " + x.listed_in + " " + x.description, axis = 1)
df["bag_of_words"] = df.apply(lambda x : x.listed_in + " " + x.description, axis = 1)
df

In [ ]:
data_train = df[["bag_of_words"]]
data_train

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorize = CountVectorizer()
train = vectorize.fit_transform(data_train["bag_of_words"])
d_train = train.toarray()
d_train

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(algorithm = "brute")
model.fit(d_train)

In [ ]:
movie_title = np.where(data_train.index == "3%")[0][0]
dt_train = data_train.iloc[movie_title,:].values
dt_train
element = vectorize.transform(dt_train).toarray()
element
distances, suggestions = model.kneighbors(n_neighbors = 10, X = element)

In [ ]:
def recommand(title):
    movie_title = np.where(data_train.index == title)[0][0]
    dt_train = data_train.iloc[movie_title,:].values
    element = vectorize.transform(dt_train).toarray()
    distances, suggestions = model.kneighbors(n_neighbors = 11, X = element)
        
    for i in range(len(suggestions)):
        if i==0:
            print("the suggestions for ",title, " are : ")
        if not i:
            for number, movie in enumerate([x for x in df.index[suggestions[i]]][1:]) :
                mov_til = np.where(data.index == movie)[0][0]
                type_ = data.iloc[mov_til, 0]
                director_ = data.iloc[mov_til, 1]
                rating_ = data.iloc[mov_til, 4]
                genre_ = data.iloc[mov_til, 5]
                print(f'{number+1}. type : {type_}, title : {movie}, director : {director_}, rating : {rating_}, genre : {genre_}')

In [ ]:
recommand([x for x in data.index.to_list() if 'Hulk Vs.' in x][0])